In [333]:
origin = "propbanksenses"

if origin == "llamasenses":
  longform_name = "pairsim_llama_llamasenses_longform.csv"
  merged_name = "pairsim_llama_llamasenses_merged.csv"
  targets_location = "/content/drive/MyDrive/projects/2023/glass_project/02_16_24/pairs_by_verb_llama_llamasenses.csv"
  raw_location = "/content/drive/MyDrive/projects/2023/glass_project/02_23_24/pair_difference_raw_llama_llamasenses_020324.csv"
elif origin == "propbanksenses":
  longform_name = "pairsim_llama_propbanksenses_longform.csv"
  merged_name = "pairsim_llama_propbanksenses_merged.csv"
  targets_location = "/content/drive/MyDrive/projects/2023/glass_project/02_02_24/pairs_by_verb_llama_propbanksenses.csv"
  raw_location = "/content/drive/MyDrive/projects/2023/glass_project/02_23_24/pair_difference_raw_llama_propbanksenses_020324.csv"
else: # for reddit items
  longform_name = "pairsim_reddit_longform.csv"
  merged_name = "pairsim_reddit_merged.csv"
  targets_location = "/content/drive/MyDrive/projects/2023/glass_project/07_07_23/pairs_by_verb_reddit_070623.csv"
  raw_location = "/content/drive/MyDrive/projects/2023/glass_project/final_data/pairdiff/raw_pairdiff_reddit.csv"

Mount drive

In [334]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import packages

In [335]:
import pandas as pd
import csv
import numpy as np
from scipy.stats import zscore


Import raw data

In [336]:
df_path = raw_location
df = pd.read_csv(df_path)
df = df.drop(columns = ["StartDate", "EndDate", "Status", "IPAddress",
                        "Progress", "Duration (in seconds)", "Finished",
                        "RecordedDate", "RecipientLastName", "RecipientFirstName",
                        "RecipientEmail", "ExternalReference", "LocationLatitude",
                        "LocationLongitude", "DistributionChannel", "UserLanguage",
                        "consent", "instructions", "Q14_1", "Q16", "Q17_1", "Q15",
                        "Q4_8", "Q5", "Q10_4", "Q9", "Scoredd", "Scored", "Serve2",
                        "Create New Field or Choose From Dropdown...", "Serve1",
                        "SC3", "SC2", "SC1", "SC0", "Q12", "Q1"])
df[:3]

,ResponseId,1_Q24_4,2_Q24_4,3_Q24_4,4_Q24_4,5_Q24_4,6_Q24_4,1_Q2_4,2_Q2_4,3_Q2_4,...,53_Q68_4,54_Q68_4,55_Q68_4,56_Q68_4,57_Q68_4,58_Q68_4,59_Q68_4,60_Q68_4,61_Q68_4,62_Q68_4
0,Response ID,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...
1,"{""ImportId"":""_recordId""}","{""ImportId"":""1_QID34_4""}","{""ImportId"":""2_QID34_4""}","{""ImportId"":""3_QID34_4""}","{""ImportId"":""4_QID34_4""}","{""ImportId"":""5_QID34_4""}","{""ImportId"":""6_QID34_4""}","{""ImportId"":""1_QID1316697397_4""}","{""ImportId"":""2_QID1316697397_4""}","{""ImportId"":""3_QID1316697397_4""}",...,"{""ImportId"":""53_QID68_4""}","{""ImportId"":""54_QID68_4""}","{""ImportId"":""55_QID68_4""}","{""ImportId"":""56_QID68_4""}","{""ImportId"":""57_QID68_4""}","{""ImportId"":""58_QID68_4""}","{""ImportId"":""59_QID68_4""}","{""ImportId"":""60_QID68_4""}","{""ImportId"":""61_QID68_4""}","{""ImportId"":""62_QID68_4""}"
2,R_1aBvEll6Ab6uy7T,1,1,1,1,100,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Import sentence pairs and ids

In [337]:
df_fillers_path = "/content/drive/MyDrive/projects/2023/glass_project/05_05_24/difference_fillers.csv"
df_fillers = pd.read_csv(df_fillers_path)
df_fillers[:3]

,pair_id,verb,sentence1,sentence2,zscore,order,block,surp1,surp2,pair_type
0,find2,find,The bee found the flower.,The court found the liability.,0.427572,4,calibration,137.058200,155.75291,different
1,line1,line,The tailor lined the coat.,The insulation lined the attic.,-0.355583,8,calibration,121.266846,207.85396,same
2,discharge2,discharge,The hospital discharged the patient.,The bank discharged the loan.,0.694240,12,calibration,133.343840,107.28486,different


In [338]:
fillers_dict = df_fillers.set_index('pair_id').to_dict(orient='index')
fillers_dict

{'find2': {'verb': 'find',
  'sentence1': 'The bee found the flower.',
  'sentence2': 'The court found the liability.',
  'zscore': 0.4275721023401325,
  'order': 4,
  'block': 'calibration',
  'surp1': 137.0582,
  'surp2': 155.75291,
  'pair_type': 'different'},
 'line1': {'verb': 'line',
  'sentence1': 'The tailor lined the coat.',
  'sentence2': 'The insulation lined the attic.',
  'zscore': -0.3555827566654691,
  'order': 8,
  'block': 'calibration',
  'surp1': 121.266846,
  'surp2': 207.85396,
  'pair_type': 'same'},
 'discharge2': {'verb': 'discharge',
  'sentence1': 'The hospital discharged the patient.',
  'sentence2': 'The bank discharged the loan.',
  'zscore': 0.6942400426384807,
  'order': 12,
  'block': 'calibration',
  'surp1': 133.34384,
  'surp2': 107.28486,
  'pair_type': 'different'},
 'throw1': {'verb': 'throw',
  'sentence1': 'The athlete threw the javelin.',
  'sentence2': 'The referee threw the flag.',
  'zscore': -0.5280450641234967,
  'order': 13,
  'block': 'ca

In [339]:
df_targets_path = targets_location
df_targets = pd.read_csv(df_targets_path)
df_targets = df_targets.drop(columns = ["Unnamed: 0"])
df_targets.rename(columns=lambda x: x.lower(), inplace=True)
df_targets[:3]

,sentence1,sentence2,pair_id,sense1,sense2,verb,pairtype
0,The man abused the dog.,The company abused the system.,abuse0,"mistreat a person or animal, often repeatedly,...",use (something) to bad effect or for a bad pur...,abuse,different
1,The man abused the dog.,The man abused the child.,abuse1,"mistreat a person or animal, often repeatedly,...",use offending language,abuse,different
2,The company abused the system.,The man abused the child.,abuse2,use (something) to bad effect or for a bad pur...,use offending language,abuse,different


In [340]:
targets_dict = df_targets.set_index('pair_id').to_dict(orient='index')
targets_dict['post6']

{'sentence1': 'The postman posted the mailboxes.',
 'sentence2': 'The poster posted the posters.',
 'sense1': 'cf report, put up for public view',
 'sense2': 'cf report, put up for public view',
 'verb': 'post',
 'pairtype': 'same'}

Calculate z-scored scores, or else just ensure that things are numeric

In [341]:
df = df.drop(columns = ["ResponseId"])
df[:3]

,1_Q24_4,2_Q24_4,3_Q24_4,4_Q24_4,5_Q24_4,6_Q24_4,1_Q2_4,2_Q2_4,3_Q2_4,4_Q2_4,...,53_Q68_4,54_Q68_4,55_Q68_4,56_Q68_4,57_Q68_4,58_Q68_4,59_Q68_4,60_Q68_4,61_Q68_4,62_Q68_4
0,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...,Rate how different the situations described by...
1,"{""ImportId"":""1_QID34_4""}","{""ImportId"":""2_QID34_4""}","{""ImportId"":""3_QID34_4""}","{""ImportId"":""4_QID34_4""}","{""ImportId"":""5_QID34_4""}","{""ImportId"":""6_QID34_4""}","{""ImportId"":""1_QID1316697397_4""}","{""ImportId"":""2_QID1316697397_4""}","{""ImportId"":""3_QID1316697397_4""}","{""ImportId"":""4_QID1316697397_4""}",...,"{""ImportId"":""53_QID68_4""}","{""ImportId"":""54_QID68_4""}","{""ImportId"":""55_QID68_4""}","{""ImportId"":""56_QID68_4""}","{""ImportId"":""57_QID68_4""}","{""ImportId"":""58_QID68_4""}","{""ImportId"":""59_QID68_4""}","{""ImportId"":""60_QID68_4""}","{""ImportId"":""61_QID68_4""}","{""ImportId"":""62_QID68_4""}"
2,1,1,1,1,100,1,1,1,1,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
df2 = df.copy()
df2 = df2.drop(1).reset_index(drop=True)

Rename columns

In [343]:
df_instance = df2.copy()

df_instance.iloc[0]
df.iloc[0].str.contains("pass7").any()

True

In [344]:
df_instance.sample(3)

,1_Q24_4,2_Q24_4,3_Q24_4,4_Q24_4,5_Q24_4,6_Q24_4,1_Q2_4,2_Q2_4,3_Q2_4,4_Q2_4,...,53_Q68_4,54_Q68_4,55_Q68_4,56_Q68_4,57_Q68_4,58_Q68_4,59_Q68_4,60_Q68_4,61_Q68_4,62_Q68_4
15,1,0,11,0,12,0,NaN,NaN,NaN,NaN,...,100,0,0,0,0,0,0,0,90,0
18,100,25,100,80,100,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,59,100,0,56,0,NaN,NaN,NaN,NaN,...,100,3,0,0,51,43,42,30,78,46


In [345]:
def rename_columns(df, target_pairs):
  df_instance = df.copy()
  all_items = set(fillers_dict.keys()) | set(target_pairs.keys())
  new_row = []
  print(df.iloc[0].str.contains("pass7").any())
  for col_name in df_instance.iloc[0]:
    # print(elem)
    try:
      col_name = col_name.split(" - ")[1]
    except:
      print("FAILED ON: " + str(col_name))

    found_item = "balanced data"
    for item in all_items:
      if item == col_name:
        found_item = item
        break
    new_row.append(found_item)

  # print(new_row)
  df_instance.loc[0] = new_row
  df_instance = df_instance.set_axis(df_instance.iloc[0], axis=1)
  # print(df_instance)
  df_instance = df_instance[2:]  # Remove the first row from the DataFrame
  # print(df_instance)
  df_instance = df_instance.drop("balanced data", axis=1)

  col_vals = pd.Series(list(df_instance.columns)).value_counts()

  cols_to_merge = col_vals[col_vals > 1]
  rem_cols = col_vals[col_vals <= 1]
  merged_dict = {}
  for col in cols_to_merge.keys():
    merged_dict[col] = df_instance[col].bfill(axis=1).iloc[:, 0]

  for col in rem_cols.keys():
    merged_dict[col] = df_instance[col]

  df_instance = pd.DataFrame(merged_dict)
  return df_instance

df_instance = rename_columns(df_instance, targets_dict)

True
FAILED ON: balanced data
FAILED ON: balanced data
FAILED ON: balanced data
FAILED ON: balanced data


In [346]:
df_instance.sample(3)

,dismiss1,permit2,design2,split1,recognize2,drop5,draw3,crash1,cover2,clear3,...,push5,post2,meet7,place0,pitch0,pinch9,pinch8,pinch3,meet8,stab0
20,0,100,98,41,100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,43,42,46,78,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51
19,58,100,16,34,60,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [348]:
all_exist = df_targets['pair_id'].isin(df_instance.columns).all()

if all_exist:
    print("All Pair_ID values exist as column names in df_instance.")
else:
    print("Not all Pair_ID values exist as column names in df_instance.")

All Pair_ID values exist as column names in df_instance.


Reshape the DataFrame to long format

In [349]:
df_instance['Rater_ID'] = df_instance.index

df_long = pd.melt(df_instance, id_vars='Rater_ID', var_name='Pair_ID', value_name='Rating')

df_long = df_long.dropna(subset=['Rating'])

# df_long

# Check if any value in "Pair_ID" is "pass7"
is_pass7_present = (df_long['Pair_ID'] == 'pass7').any()

# Print the result
if is_pass7_present:
    print("The value 'pass7' is present in the 'Pair_ID' column.")
else:
    print("The value 'pass7' is not present in the 'Pair_ID' column.")


The value 'pass7' is present in the 'Pair_ID' column.


In [350]:
df_long.rename(columns=lambda x: x.lower(), inplace=True)

Insert sentence information and filler/calibration/target information

In [354]:
df_long["sentence_type"] = df_long["pair_id"].apply(lambda x: "calibration" if x in fillers_dict.keys() else "target")
df_long["verb"] = df_long["pair_id"].apply(lambda x: targets_dict[x]["verb"] if x in targets_dict else fillers_dict[x]["verb"])
df_long["pair_type"] = df_long["pair_id"].apply(lambda x: targets_dict[x]["pairtype"] if x in targets_dict else fillers_dict[x]['pair_type'])


def get_values(key, dictionary):
    if key in dictionary:
        return dictionary[key]["sentence1"], dictionary[key]["sentence2"]
    else:
        return None, None

def get_sense_values(key, dictionary):
    if key in dictionary:
        return dictionary[key]["sense1"], dictionary[key]["sense2"]
    else:
        return None, None

df_long[["sentence1", "sentence2"]] = df_long["pair_id"].apply(lambda x: get_values(x, fillers_dict) if x in fillers_dict else get_values(x, targets_dict)).apply(pd.Series)
df_long[["sense1", "sense2"]] = df_long["pair_id"].apply(lambda x: ("calibration", "calibration") if x in fillers_dict else get_sense_values(x, targets_dict)).apply(pd.Series)

columns = df_long.columns
lowercase_columns = [col.lower() for col in columns]
df_long.columns = lowercase_columns

df_long.sample(3)

,rater_id,pair_id,rating,sentence_type,verb,pair_type,sentence1,sentence2,sense1,sense2
2970,14,shoot5,100,target,shoot,different,The director shot the scene.,The bullet shot the man.,record on film,move very rapidly
7865,8,fire8,9,target,fire,same,The baker fired the breads.,The firemen fired the kiln.,cause to burn or bake,cause to burn or bake
1654,3,save1,80,target,save,different,The babysitter saved the children.,The boy saved the coins.,keep from spending,"collect, accrue, save up (non phrasal, though)"


In [161]:
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [162]:
def single_sbert(sent1, sent2):
  sentences = [sent1, sent2]
  embeddings = model.encode(sentences)
  similarity = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))
  return float(similarity[0][0])


In [163]:
pair_id_sbert_dict = {x: float('NaN') for x in list(fillers_dict.keys()) + list(targets_dict.keys())}
for x in pair_id_sbert_dict.keys():
  if x in fillers_dict.keys():
    pair_id_sbert_dict[x] = single_sbert(fillers_dict[x]["Sentence1"], fillers_dict[x]["Sentence2"])
  else:
    pair_id_sbert_dict[x] = single_sbert(targets_dict[x]["Sentence1"], targets_dict[x]["Sentence2"])

# pair_id_sbert_dict

In [164]:
df_long["sbert"] = df_long["pair_id"].apply(lambda x: pair_id_sbert_dict[x])
df_long.sample(3)

,rater_id,pair_id,rating,origin,verb,pair_type,sentence1,sentence2,sense1,sense2,sbert
8261,27,meet0,0,target,meet,different,The team met the objective.,The youngest met the queen.,"arrive at, achieve","kennenlernen-- come upon, become acquainted wi...",0.101741
6026,25,discharge2,13,filler,discharge,filler,The hospital discharged the patient.,The bank discharged the loan.,filler,filler,0.493226
1654,3,save1,80,target,save,different,The babysitter saved the children.,The boy saved the coins.,keep from spending,"collect, accrue, save up (non phrasal, though)",0.418172


In [165]:
df_long.to_csv(longform_name, index=False)